# 算法学习VPG 2020/3/17

In [8]:
import tensorflow as tf
import gym
import numpy as np

env = gym.make('CartPole-v0')

obs_shape = env.observation_space.shape
act_shape = env.action_space.shape

print("observation shape", obs_shape)
print("action shape", act_shape)

observation shape (4,)
action shape ()


In [ ]:
obs_ph = tf.placeholder(dtype=tf.float32, shape=obs_shape)
act_ph = tf.placeholder(dtype=tf.int32, shape=(None,))
R_ph = tf.placeholder(dtype=tf.float32, shape=None)

layer1 = tf.layers.dense(obs_ph[np.newaxis,:], activation=tf.tanh, units=32)
layer2 = tf.layers.dense(layer1, activation=tf.tanh, units=32)
out = tf.layers.dense(layer2, activation=None, units=env.action_space.n)

In [19]:
logp_all = tf.nn.log_softmax(out)
pi = tf.squeeze(tf.multinomial(out,1), axis=1) #choose an action at state now
logp = tf.reduce_sum(tf.one_hot(act_ph, depth=env.action_space.n) * logp_all, axis=1) #log p of choosing this action

pi_loss = -tf.reduce_mean(logp * R_ph)
train_pi = tf.train.AdamOptimizer(learning_rate=3e-4).minimize(pi_loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(1000):
    obs = env.reset()
    for step in range(epoch):
        act = sess.run(pi, feed_dict={obs_ph:obs})
        ob2, r, d, _ = env.step(act[0])
        env.render()
        sess.run(train_pi, feed_dict={obs_ph:obs, act_ph:act, R_ph:r})
        if d:
            break
        
    